In [1]:
# general imports
import numpy as np
import pandas as pd
from urllib.request import urlopen
import json
import os
import pathlib
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# pandas formatting 
pd.set_option("display.max_rows", 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:

# load county information for plotly from saved file 
with open('counties.json','r') as jin:
    counties = json.load(jin)



In [4]:

def FIPS_function(row):
    state = str(row['State Code']).zfill(2)
    county = str(row['County Code']).zfill(3)
    return str(state + county)
# convert to NO2 ug/m^3 for reference
def no2_mass_by_vol(ppb):
    ugm3 = 1.88*ppb
    return ugm3



def set_daily_cases_deaths(df):
    df['daily_new_cases'] = df['JHU_ConfirmedCases.data'].diff()
    df['daily_new_deaths'] = df['JHU_ConfirmedDeaths.data'].diff()
    return df
# plotting one day's avg 
def show_day_mean(df, date):
    fig = px.choropleth(df[df['Date Local']==date], geojson=counties, locations='fips', color='Arithmetic Mean',
                               color_continuous_scale="Plasma",
                               range_color=(0, 70), #max value for daily avg is ~60ppb
                               scope="usa",
                               labels={'Arithmetic Mean':'Arithmetic Mean (ppb)'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":1,'autoexpand':True })
    fig.update_layout(
        autosize=False,
        width=1200,
        height=900,
    )
    return fig


# plotting one day's max value
def show_day_max(df, date):
    fig = px.choropleth(df[df['Date Local']==date], geojson=counties, locations='fips', color='1st Max Value',
                               color_continuous_scale="Plasma",
                               range_color=(0, 70), #max value for daily avg is ~60ppb
                               scope="usa",
                               labels={'1st Max Value':'1st Max Value (ppb)'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":1,'autoexpand':True })
    fig.update_layout(
        autosize=False,
        width=1200,
        height=900,
    )
    return fig   
def show_sites(df):
    fig = px.choropleth(df, geojson=counties, locations='fips', color='Parameter Code',
                               color_continuous_scale="Plasma",
                               range_color=(0, 1), #max value for daily avg is ~60ppb
                               scope="usa",
                               labels={'1st Max Value':'1st Max Value (ppb)'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":1,'autoexpand':True })
    fig.update_layout(
        autosize=False,
        width=1200,
        height=900,
    )
    return fig

# Loading EPA Data

In [5]:
cwd = pathlib.Path.cwd()
air_data_dir = cwd.joinpath("data","air_quality")


In [6]:
# no2_20 = pd.read_csv("./data/air_quality/no2/daily_no2_2020_with_FIPS.csv", dtype={'fips':'string'})
no2_20 = pd.read_csv(air_data_dir.joinpath("no2","daily_no2_2020_with_FIPS.csv"), dtype={'fips':'string'})
# covid1 = pd.read_csv(cwd.joinpath("data","covid",","daily_no2_2020_with_FIPS.csv"), dtype={'fips':'string'})

#### Argument for removing columns
 
We are looking for a link between 2 datasets, for now we will assume things like the 'POC' field (This is the “Parameter Occurrence Code” used to distinguish different instruments that measure the same parameter at the same site.) are unimportant to our investigation.

Similarly we will remove the method code, method name, ect.

more information on the field definitions can be found here -> https://aqs.epa.gov/aqsweb/airdata/FileFormats.html#_content_4

In [7]:
no2_columns = ['Date Local', 'fips', 'Arithmetic Mean','1st Max Value', '1st Max Hour',
               'AQI', 'Units of Measure', 'Event Type', 'Mean ugm3', 'Site Num', 'Observation Count',
               'Observation Percent', 'Longitude', 'Latitude', 'Site Num','Local Site Name', 'Address', 
               'State Name', 'County Name', 'CBSA Name', 'State Code', 'County Code']
slim_no2 = no2_20[no2_columns]
slim_no2.head()

,Date Local,fips,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Units of Measure,Event Type,Mean ugm3,Site Num,Observation Count,Observation Percent,Longitude,Latitude,Site Num,Local Site Name,Address,State Name,County Name,CBSA Name,State Code,County Code
0,2020-01-01,01073,15.752381,28.2,6,26,Parts per billion,None,29.614476,23,21,88.0,-86.815,33.553056,23,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,"Birmingham-Hoover, AL",1,73
1,2020-01-02,01073,9.595833,16.3,10,15,Parts per billion,None,18.040166,23,24,100.0,-86.815,33.553056,23,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,"Birmingham-Hoover, AL",1,73
2,2020-01-03,01073,17.300000,31.6,10,29,Parts per billion,None,32.524000,23,24,100.0,-86.815,33.553056,23,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,"Birmingham-Hoover, AL",1,73
3,2020-01-04,01073,2.791667,7.5,23,7,Parts per billion,None,5.248334,23,24,100.0,-86.815,33.553056,23,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,"Birmingham-Hoover, AL",1,73
4,2020-01-05,01073,14.408333,34.0,19,32,Parts per billion,None,27.087666,23,24,100.0,-86.815,33.553056,23,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,"Birmingham-Hoover, AL",1,73


In [8]:
no2df = slim_no2.set_index(['Date Local','fips'])

In [9]:

# multiple sites in one county
slim_no2.loc[(slim_no2['Date Local']=='2020-01-01') & (slim_no2['fips']=='06001')]

,Date Local,fips,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Units of Measure,Event Type,Mean ugm3,Site Num,Observation Count,Observation Percent,Longitude,Latitude,Site Num,Local Site Name,Address,State Name,County Name,CBSA Name,State Code,County Code
4002,2020-01-01,06001,7.865217,18.5,1,17,Parts per billion,None,14.786608,7,23,96.0,-121.784217,37.687526,7,Livermore,793 Rincon Ave.,California,Alameda,"San Francisco-Oakland-Hayward, CA",6,1
4368,2020-01-01,06001,12.617391,21.3,0,20,Parts per billion,None,23.720695,9,23,96.0,-122.169935,37.743065,9,Oakland,9925 International Blvd,California,Alameda,"San Francisco-Oakland-Hayward, CA",6,1
4733,2020-01-01,06001,15.921739,28.0,21,26,Parts per billion,None,29.932869,11,23,96.0,-122.282347,37.814781,11,Oakland West,1100 21st Street,California,Alameda,"San Francisco-Oakland-Hayward, CA",6,1
5096,2020-01-01,06001,15.934783,23.2,21,22,Parts per billion,None,29.957392,12,23,96.0,-122.263376,37.793624,12,Laney College,Laney College Eighth St. parking lot Aisle J,California,Alameda,"San Francisco-Oakland-Hayward, CA",6,1
5462,2020-01-01,06001,12.826087,23.5,0,22,Parts per billion,None,24.113044,13,23,96.0,-122.302741,37.864767,13,Berkeley Aquatic Park,1 Bolivar Dr,California,Alameda,"San Francisco-Oakland-Hayward, CA",6,1
5828,2020-01-01,06001,10.808696,18.7,0,17,Parts per billion,None,20.320348,15,23,96.0,-121.903019,37.701222,15,Pleasanton - Owens Ct,Owens Ct.,California,Alameda,"San Francisco-Oakland-Hayward, CA",6,1


In [29]:
date_only=slim_no2.groupby(['Date Local'])
date_fips=slim_no2.groupby(['Date Local','fips'])

In [37]:
# date_only.
# date_fips.get_group(('2020-01-01', '01073')
county_mean=date_fips.agg({'Arithmetic Mean':'mean'})

In [44]:
# county_mean[(county_mean['Date Local']=='2020-01-01')&(county_mean['fips']=='01073')]
county_mean.loc[('2020-01-01','01073')]

Arithmetic Mean    16.373918
Name: (2020-01-01, 01073), dtype: float64

In [11]:
the_first = no2_20[(no2_20['Date Local']=='2020-01-01')&(no2_20['fips']=='01073')]

In [12]:
the_first

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,fips,Mean ugm3
0,1,73,23,42602,1,33.553056,-86.815000,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour,2020-01-01,Parts per billion,None,21,88.0,15.752381,28.2,6,26,200,Teledyne-API Model 200EUP or T200UP - Photolyt...,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-02-25,01073,29.614476
362,1,73,2059,42602,1,33.521427,-86.844112,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour,2020-01-01,Parts per billion,None,22,92.0,16.995455,31.0,18,29,200,Teledyne-API Model 200EUP or T200UP - Photolyt...,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-02-25,01073,31.951455


### Multiple sample sites

Some counties have multiple air monitoring stations. Below code builds a dictionary whose keys are the fips codes with multiple sites.

In [45]:
all_fips = list(no2_20['fips'].unique())
county_site_count = {}
for code in all_fips:
    site_count = no2_20[(no2_20['Date Local']=='2020-01-01')&(no2_20['fips']== code)]['Site Num'].count()
    if site_count > 1:
        county_site_count[code] = site_count

For a quick naive solution we can average the averages for each county, then take the max recorded across the counties.

In [55]:
no2_20.iloc[0]['Event Type']

'None'

In [48]:
def avg_county_pollution(df, column_to_avg = "Arithmetic Mean", date_column=None):
    
    
    if date_column == None:
        date_column = 'Date Local'
    # All days 
    # days = list(df[date_column].unique())
    # all_fips = list(df['fips'].unique())
    avg_by_day = df.groupby(['Date Local','fips']).agg({'Arithmetic Mean': 'mean','1st Max Value':'max','AQI':'max'})
    return avg_by_day
    

In [51]:
squashed_no2 = avg_county_pollution(no2_20)

In [59]:
squashed_no2.to_pickle(air_data_dir.joinpath("no2","daily_2020_multi_index.pkl"))

# exploring Covid data

In [5]:
abbeville = pd.read_pickle(cwd.joinpath("data","covid","processed_data","county_merged_parts","Abbeville_SouthCarolina_UnitedStates.pkl"))

In [10]:
abbeville.shape

(654, 62)

In [11]:
non_zero_columns = [c for c in abbeville.columns if (abbeville[c].sum() > 0)]

non_zero_columns

['JHU_ConfirmedCases.data',
 'NYT_ConfirmedCases.data',
 'NYT_ConfirmedCases.missing',
 'JHU_ConfirmedDeaths.data',
 'JHU_ConfirmedRecoveries.missing',
 'NYT_AllCausesDeathsWeekly_Expected_Deaths_AllCauses.missing',
 'NYT_ConfirmedDeaths.data',
 'NYT_ConfirmedDeaths.missing',
 'NYT_AllCausesDeathsWeekly_Excess_Deaths.missing',
 'NYT_AllCausesDeathsWeekly_Deaths_AllCauses.missing',
 'NYT_AllCausesDeathsMonthly_Deaths_AllCauses.missing',
 'NYT_AllCausesDeathsMonthly_Excess_Deaths.missing',
 'NYT_AllCausesDeathsMonthly_Expected_Deaths_AllCauses.missing',
 'TotalPopulation.data',
 'MaleAndFemale_AtLeast65_Population.data',
 'Male_Total_Population.data',
 'Female_Total_Population.data',
 'MaleAndFemale_Under18_Population.data',
 'BLS_EmployedPopulation.data',
 'BLS_UnemployedPopulation.data',
 'BLS_UnemploymentRate.data',
 'BLS_LaborForcePopulation.data',
 'AverageDailyTemperature.data',
 'AverageDewPoint.data',
 'AverageRelativeHumidity.data',
 'AverageSurfaceAirPressure.data',
 'AveragePr

### Slimming down the census data

There are things that are not needed at this stage and if needed later could be looked up by county. I will remove the ones that don't seem to contribute much and see if a merge with no2 is reasonable

In [43]:
# Super slimmed down, no NYT, no *.missing
less_cols=['fips','JHU_ConfirmedCases.data','NYT_ConfirmedCases.data','JHU_ConfirmedDeaths.data','JHU_ConfirmedRecoveries.data',
           'TotalPopulation.data','MaleAndFemale_AtLeast65_Population.data','Male_Total_Population.data','Female_Total_Population.data',
           'MaleAndFemale_Under18_Population.data','BLS_EmployedPopulation.data','BLS_UnemployedPopulation.data','BLS_UnemploymentRate.data',
           'BLS_LaborForcePopulation.data','AverageDailyTemperature.data','hospitalIcuBeds','hospitalStaffedBeds','hospitalLicensedBeds']
df_small=abbeville[less_cols].reset_index()


In [45]:
ab_df = df_small.set_index(['dates','fips'])
ab_df.head()

,,JHU_ConfirmedCases.data,NYT_ConfirmedCases.data,JHU_ConfirmedDeaths.data,JHU_ConfirmedRecoveries.data,TotalPopulation.data,MaleAndFemale_AtLeast65_Population.data,Male_Total_Population.data,Female_Total_Population.data,MaleAndFemale_Under18_Population.data,BLS_EmployedPopulation.data,BLS_UnemployedPopulation.data,BLS_UnemploymentRate.data,BLS_LaborForcePopulation.data,AverageDailyTemperature.data,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds
dates,fips,,,,,,,,,,,,,,,,,
2020-01-01,45001.0,0.0,0.0,0.0,0.0,24527.0,5343.0,11868.0,12673.0,4924.0,9706.5,370.5,3.676689,10077.0,44.875000,6.0,25.0,25.0
2020-01-02,45001.0,0.0,0.0,0.0,0.0,24527.0,5343.0,11868.0,12673.0,4924.0,9706.5,370.5,3.676689,10077.0,45.843750,6.0,25.0,25.0
2020-01-03,45001.0,0.0,0.0,0.0,0.0,24527.0,5343.0,11868.0,12673.0,4924.0,9706.5,370.5,3.676689,10077.0,51.935020,6.0,25.0,25.0
2020-01-04,45001.0,0.0,0.0,0.0,0.0,24527.0,5343.0,11868.0,12673.0,4924.0,9706.5,370.5,3.676689,10077.0,54.208333,6.0,25.0,25.0
2020-01-05,45001.0,0.0,0.0,0.0,0.0,24527.0,5343.0,11868.0,12673.0,4924.0,9706.5,370.5,3.676689,10077.0,41.916667,6.0,25.0,25.0


In [49]:
all_no2_fips = list(no2_20['fips'].unique())
len(all_no2_fips)

260

In [194]:
list(no2_20['fips'].unique())

['01073',
 '04013',
 '04019',
 '05035',
 '05119',
 '06001',
 '06007',
 '06013',
 '06019',
 '06023',
 '06025',
 '06029',
 '06031',
 '06037',
 '06039',
 '06041',
 '06047',
 '06053',
 '06055',
 '06059',
 '06061',
 '06065',
 '06067',
 '06071',
 '06073',
 '06075',
 '06077',
 '06079',
 '06081',
 '06083',
 '06085',
 '06095',
 '06097',
 '06099',
 '06101',
 '06107',
 '06111',
 '06113',
 '08001',
 '08007',
 '08029',
 '08031',
 '08045',
 '08059',
 '08067',
 '08103',
 '08123',
 '09001',
 '09003',
 '09009',
 '10003',
 '11001',
 '12011',
 '12031',
 '12057',
 '12086',
 '12095',
 '12103',
 '13089',
 '13121',
 '15003',
 '15007',
 '16001',
 '17031',
 '17117',
 '17163',
 '18089',
 '18097',
 '18141',
 '18163',
 '19153',
 '19177',
 '20133',
 '20173',
 '20191',
 '20195',
 '20209',
 '21019',
 '21037',
 '21059',
 '21067',
 '21111',
 '21145',
 '22005',
 '22019',
 '22033',
 '22047',
 '22051',
 '22063',
 '22071',
 '22121',
 '23003',
 '23005',
 '23011',
 '24005',
 '24023',
 '24027',
 '24033',
 '24510',
 '25009',
